# CALFEWS: California Food-Energy-Water System simulation model

This notebook will rerun all simulations and recreate all figures from the following manuscript:

Zeff, H.B., Hamilton, A.L., Malek, K., Herman, J.D., Cohen, J.S., Medellin-Azuara, J., Reed, P.M., and G.W. Characklis. (2020). "California's Food-Energy-Water System: An Open Source Simulation Model of Adaptive Surface and Groundwater Management in the Central Valley." (In review, preprint available at https://engrxiv.org/sqr7e/).

Prior to running this notebook, follow the instructions in "README.md" to download the model and data, and install all Python dependencies.

## Run validation 

First, make sure you do not already have output stored at ``results/Historical_validation_1997-2016`` or ``results/Historical_reconstruction_1906-2016``. If so, it needs to be moved or deleted, or it will cause an error.

Override parameter file with version in "runtime_params_validation.ini"

In [1]:
! cp runtime_params_validation.ini runtime_params.ini

Run validation mode (1997-2016). This will take ~7-10 minutes. Note: you may need to change ``python`` to ``python3`` depending on your system setup.

In [2]:
results_validation = "results/Historical_validation_1997-2016"
! python -W ignore run_main_cy.py $results_validation

#######################################################
Begin initialization...
Initialization complete,  0:00:22.598834
Begin simulation,  0:00:22.598834
results/Historical_validation_1997-2016
Year  1.0 ,  0:00:46.108681
Year  2.0 ,  0:01:10.661066
Year  3.0 ,  0:01:32.151072
Year  4.0 ,  0:01:53.868749
Year  5.0 ,  0:02:15.115098
Year  6.0 ,  0:02:34.137175
Year  7.0 ,  0:02:52.687703
Year  8.0 ,  0:03:10.650292
Year  9.0 ,  0:03:31.159241
Year  10.0 ,  0:03:54.050718
Year  11.0 ,  0:04:12.205817
Year  12.0 ,  0:04:29.397780
Year  13.0 ,  0:04:46.375171
Year  14.0 ,  0:05:04.668154
Year  15.0 ,  0:05:27.369661
Year  16.0 ,  0:05:46.593083
Year  17.0 ,  0:06:04.518041
Year  18.0 ,  0:06:20.759474
Year  19.0 ,  0:06:35.704124
Year  20.0 ,  0:06:52.808104
Simulation complete, 0:07:03.915143


## Run simulation 
Now change to simulation mode (1906-2016) and rerun. This will take ~35-50 minutes.

In [3]:
results_simulation = "results/Historical_reconstruction_1906-2016"
! sed -i 's:\"validation\":\"simulation\":' runtime_params.ini
! python -W ignore run_main_cy.py $results_simulation

#######################################################
Begin initialization...
Initialization complete,  0:01:24.866315
Begin simulation,  0:01:24.866315
results/Historical_reconstruction_1906-2016
Year  1.0 ,  0:01:51.978212
Year  2.0 ,  0:02:14.948912
Year  3.0 ,  0:02:35.572043
Year  4.0 ,  0:02:57.608460
Year  5.0 ,  0:03:18.695877
Year  6.0 ,  0:03:37.710828
Year  7.0 ,  0:03:54.001180
Year  8.0 ,  0:04:09.644005
Year  9.0 ,  0:04:27.211545
Year  10.0 ,  0:04:43.867081
Year  11.0 ,  0:05:02.884200
Year  12.0 ,  0:05:20.825918
Year  13.0 ,  0:05:37.099548
Year  14.0 ,  0:05:52.991596
Year  15.0 ,  0:06:08.628694
Year  16.0 ,  0:06:24.806240
Year  17.0 ,  0:06:41.623526
Year  18.0 ,  0:06:58.678315
Year  19.0 ,  0:07:13.881063
Year  20.0 ,  0:07:29.971106
Year  21.0 ,  0:07:45.893354
Year  22.0 ,  0:08:02.761517
Year  23.0 ,  0:08:18.794074
Year  24.0 ,  0:08:34.113534
Year  25.0 ,  0:08:49.404327
Year  26.0 ,  0:09:04.400322
Year  27.0 ,  0:09:20.382019
Year  28.0 ,  0:09:35.94079

## Post-processing
Run post-processing to analyze and visualize results. This will take a few hours.

In [5]:
results_figs = 'results/figs'
results_sankeys = 'results/sankeys'
! python -W ignore modeling_paper_postprocess.py $results_validation $results_simulation $results_figs $results_sankeys

Finish set up,  0:00:21.617461
Finish Delta pumping/outflow figure,  0:02:50.287437
Finish district deliveries figure,  0:26:28.575748
Finish snowpack/flow relationship figure,  0:33:03.686595
Finish state response figure,  0:33:07.263782
Finish validation figures,  0:33:18.926046
Starting Sankey diagrams (this will take a while)
14600 September
14601 September
14602 September
14603 September
14604 September
14605 September
14606 September
14607 September
14608 September
14609 September
14610 October
14611 October
14612 October
14613 October
14614 October
14615 October
14616 October
14617 October
14618 October
14619 October
14620 October
14621 October
14622 October
14623 October
14624 October
14625 October
14626 October
14627 October
14628 October
14629 October
14630 October
14631 October
14632 October
14633 October
14634 October
14635 October
14636 October
14637 October
14638 October
14639 October
14640 October
14641 November
14642 November
14643 November
14644 November
14645 November

Show figures from postprocessing scripts. Note that if you rerun this, you may need to save your changes and then refresh the browser for the figures below to update to their newest versions.

In [17]:
from IPython.display import HTML
HTML('<img src="' + results_figs + '/delta_pumping_extended_simulation.png">')

In [18]:
HTML('<img src="' + results_figs + '/district_water_use_annual_validation.png">')

In [19]:
HTML('<img src="' + results_figs + '/district_water_use_annual_simulation.png">')

In [20]:
HTML('<img src="' + results_figs + '/state_estimation_publication.png">')

In [21]:
HTML('<img src="' + results_figs + '/state_response_sanluisstate_wheeler.png">')

In [22]:
HTML('<img src="' + results_figs + '/model_validation_sierra.png">')

In [23]:
HTML('<img src="' + results_figs + '/model_validation_delta.png">')

In [24]:
HTML('<img src="' + results_figs + '/model_validation_sanluis.png">')

In [25]:
HTML('<img src="' + results_figs + '/model_validation_bank.png">')

In [27]:
HTML('<img src="' + results_sankeys + '/cali_sankey1946.gif">')